## Jass Style with ML

Problem: Find jass style based on 9 cards

Card suits: Rose, Shield, Oak, Bell

Card values: Ace, King, Queen, Jack, Ten, Nine, Eight, Seven, Six

Jass Styles: Top Down, Bottom Up, Trump Rose, Trump Shield, Trump Oak, Trump Bell

Input Value: 4 Bits One Hot Suit, 9 Bits One Hot Value
Output: 6 Bit One Hot Style

### Import data
Read data from CSV files and push them into a Numpy array

In [2]:
import csv
import urllib.request
import tensorflow as tf
import numpy as np

features = []
labels = []

for line in urllib.request.urlopen('https://www.dropbox.com/s/vwu6d5kd4oysqj6/jass100k.csv?dl=1'):
    feature, label = str(line).split(';')
    features.append(list(feature[2:]))
    labels.append(list(label[:-3]))

features = np.stack(features)
labels = np.stack(labels)

print(features.size)
print(labels.size)

11700000
600000


### NN model by keras
Very simple model

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=117))
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))

Using TensorFlow backend.


In [4]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [9]:
model.fit(features, labels, validation_split = 0.1)

Train on 90000 samples, validate on 10000 samples
Epoch 1/1
90000/90000 [==============================] - 8s 86us/step - loss: 0.1506 - acc: 0.9436 - val_loss: 0.1347 - val_acc: 0.9488


In [30]:
def cardsInput(cards):
    result = []
    suits = {
        'r': [1,0,0,0],
        's': [0,1,0,0],
        'o': [0,0,1,0],
        'b': [0,0,0,1]
    }
    values = {
        'a': [1,0,0,0,0,0,0,0,0],
        'k': [0,1,0,0,0,0,0,0,0],
        'q': [0,0,1,0,0,0,0,0,0],
        'j': [0,0,0,1,0,0,0,0,0],
        '10': [0,0,0,0,1,0,0,0,0],
        '9': [0,0,0,0,0,1,0,0,0],
        '8': [0,0,0,0,0,0,1,0,0],
        '7': [0,0,0,0,0,0,0,1,0],
        '6': [0,0,0,0,0,0,0,0,1]
    }
    for card in cards:
        result.extend(suits[card[0:1]])
        result.extend(values[card[1:]])
        
    return np.array([result])

In [24]:
prediction[0]

array([  5.79788617e-09,   4.51999514e-11,   3.88790866e-09,
         9.99960184e-01,   3.97561707e-05,   1.61103877e-08], dtype=float32)

In [25]:
def styleOutput(prediction):
    result = ['Top Down', 'Bottom Up', 'Trump Rose', 'Trump Shield', 'Trump Oak', 'Trump Bell']
    return result[np.argmax(prediction[0])]


In [29]:
styleOutput(model.predict(cardsInput(['sa','sk','sq','oj','s10','o9','s8','s7','s6'])))

'Trump Oak'